# Exploring the Available Files

Now that we know a bit more about the ePIC software stack and the different stages involved, we will take a closer look at the simulation campaign files that are already available for us.

In [ ]:
using XRootD.XrdCl

eic_server = "root://dtn-eic.jlab.org/"
fs = FileSystem(eic_server)

## Folder Structure

The simulation campaign files stored on the xrootd server are sorted in a fairly logical manner. In this section, we will explore this folder structure and take a look at how we can browse the files stored on this server.

### Exploring the Structure

To begin with, files relating to ePIC are stored under -

/volatile/eic/EPIC/
We can take a look at what is in this directory -

In [ ]:
           
st, entries = readdir(fs, "/volatile/eic/EPIC/")     # ls the directory
isError(st) && error(st)                             # check for errors
for f in entries
    println(f)                                       # print each entry 
end

So, a few different folders. There are three we will focus on though

- EVGEN: The input hepmc3 datasets
  E.g. some files that have been supplied by a physics event generator
- FULL: The full GEANT4 output root files (usually only saved for a fraction of runs)
  If running a simulation yourself, this would be your output from processing npsim
- RECO: The output root files from the reconstruction
  And again, if running yourself, this would be your output from EICrecon

Let's take a look in RECO

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO")     # ls the directory
isError(st) && error(st)                             # check for errors
for f in entries
    println(f)                                       # print each entry 
end

This is a list of different simulation campaigns. These are formatted as `YY.MM.Campaign`.
So 25.08.0 is the Augu 2025 simulation campaign. Some files will not be run in .0 campaigns, see the simulation campaign pages for specifics.

For now, we'll jump into `25.08.0` with the detector geometry `craterlake`

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake")     # ls the directory
isError(st) && error(st)                             # check for errors
for f in entries
    println(f)                                       # print each entry 
end

	
We now have distinct categories of physics simulations that have been processed -

- BACKGROUNDS: Background events/reactions
- SINGLE: Single particle simulations (typically to check specific detector response to 
  a specific particle)
- SIDIS: Semi-Inclusive Deep Inelastic Scattering
- DIS: Deep Inelastic Scattering
- EXCLUSIVE: Exclusive physics processes/reactions

The file we downloaded previously was a DIS reaction, so let's jump in there 

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS")     # ls the directory
isError(st) && error(st)                             # check for errors
for f in entries
    println(f)                                       # print each entry 
end

Just three directories -

- CC: Charged current
- NC: Neutral current
- BeAGLE1.03.02-1.0: A specific event genearator (BeAGLE) version

Again, we had NC previously so let's follow that -

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC")
isError(st) && error(st)
for f in entries
    println(f)
end

Again, three directories. This time, nice and straightforward, these are just the beam energy combinations that were processed. Typically, these are expressed as -

- ElectronBeamEnergy x Proton(Ion) Beam Energy
    - 18x275  - 18 GeV electrons on 275 GeV protons
    - 10x100  - 10 GeV electrons on 100 GeV protons
    - 5x41    - 5 GeV electrons on 41 GeV protons

And again, we previously downloaded an 18x275 file, so let's proceed with that -

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275")
isError(st) && error(st)
for f in entries
    println(f)
end

4 choices, this time, these correspond to the minimum Q2 value (4-monentum exchange squared) in the simulated files. Let's check that, this time though we'll only print the first 10 entries

In [ ]:
st, entries = readdir(fs, "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275/minQ2=10")
for f in sort(entries)[1:10]
    println(f)
end
println("There are: ",length(entries)," files in total in directory - /volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275/minQ2=10")

Check the size of the first N files

In [ ]:
totalsize=0
files = sort(entries)
N=10
for entry in files[1:N]
    fname = "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275/minQ2=10/" * entry
    println(entry," File Size: ",(((stat(fs,fname))[2]).size)/(1024^2), " MB")
    totalsize += (((stat(fs,fname))[2]).size)/(1024^2)
end
println("\nSize of all ", N, " files is: ", totalsize, " MB\n")

## Copy one of the files locally
This will be used for speeding up testing 

In [ ]:
f = File()
fname = "/volatile/eic/EPIC/RECO/25.08.0/epic_craterlake/DIS/NC/18x275/minQ2=10/" * files[1]

# Read the remote file and fill a buffer
st, _ = open(f, eic_server * fname, OpenFlags.Read)
st, readbuffer = read(f, stat(fs,fname)[2].size)
close(f)

# Write the buffer to a local file
open(files[1], "w") do localfile
    write(localfile, readbuffer)
end